In [1]:
from pyfiles import (parametrics as par,
    complex as cplx,
    matrix as mat,
    vineyard as vin,
    input as inp,
    plot as ourplot,
    grid as grid,
    utils as utils,
    examples as ex,
    sneaky_matrix as sneaky
)
from pyfiles.examples import CameraOpt

# so we don't have to reboot kernel every time we modify classes.py
%load_ext autoreload
%autoreload 2
%env RUST_BACKTRACE=1

import numpy as np
from copy import deepcopy

from typing import *

import matplotlib.pyplot as plt
%matplotlib inline

import mars

env: RUST_BACKTRACE=1


In [4]:
filename = 'input/hinge.obj'

class vineyard2(vin.vineyard):
  def __init__(self, *args, **kwargs):
    super().__init__(*args, **kwargs)
    self.swaps = []

  def get_point_key(self, point: np.ndarray):
    return None
  
  def prune(self, s1: cplx.simplex, s2: cplx.simplex, old_state, new_state):
    if s1.dim() == s2.dim():
      self.swaps.append((s1, s2, old_state, new_state))
    return True

p0 = [0.3, 0.3, 0.3]
p1 = [0.6, 0.3, 0.3]


py_complex = inp.read_obj(filename)
py_vineyard = vineyard2(py_complex)
py_state0 = py_vineyard.reduce(p0)
py_state1 = py_vineyard.reduce_vine(py_state0, p1)

print('Python swaps')
print(f'#swaps = {len(py_vineyard.swaps)}')
print('   '.join([f'{a.prettyrepr()}-{b.prettyrepr()}' for (a, b, _, _) in py_vineyard.swaps]))


rs_complex = mars.read_from_obj(filename)
rs_state0 = mars.reduce_from_scratch(rs_complex, p0)
(rs_state1, swaps) = mars.vineyards_123(rs_complex, rs_state0, p1)

sdict = {0: 'v', 1: 'e', 2: 'f'}
print()
print('Rust swaps') 
print(f'#swaps = {len(swaps)}')
print('   '.join([f'{sdict[dim]}{a}-{sdict[dim]}{b}' for (dim, (a, b)) in swaps]))


other_order
[0, 1, 4, 5, 6, 8, 9, 11, 2, 7, 3, 10]
[simplex ∅ bd [], simplex v0 bd [-1], simplex v1 bd [-1], simplex v3 bd [-1], simplex e0 bd [0, 1], simplex e3 bd [1, 3], simplex e4 bd [3, 0], simplex t1 bd [0, 3, 4], simplex v2 bd [-1], simplex e1 bd [1, 2], simplex e2 bd [2, 0], simplex t0 bd [0, 1, 2]]
Python swaps
#swaps = 1
e2-e1
Distances in compute_permutations: (key_point=[0.3, 0.3, 0.3])
[FloatOrd(0.6699999999999999), FloatOrd(0.6699999999999999), FloatOrd(1.0699999999999998), FloatOrd(0.6699999999999999)]
[FloatOrd(0.6699999999999999), FloatOrd(1.0699999999999998), FloatOrd(1.0699999999999998), FloatOrd(0.6699999999999999), FloatOrd(0.6699999999999999)]
[FloatOrd(1.0699999999999998), FloatOrd(0.6699999999999999)]

Rust swaps
#swaps = 2
e3-e4   e0-e4
Distances in compute_permutations: (key_point=[0.6, 0.3, 0.3])
[FloatOrd(0.33999999999999997), FloatOrd(0.9399999999999998), FloatOrd(0.7399999999999999), FloatOrd(0.94)]
[FloatOrd(0.9399999999999998), FloatOrd(0.939999999999999

In [2]:
cplx = mars.read_from_obj('input/three-points.obj')

pt0 = [0.8, 0.55, 0]
pt1 = [0.8, 0.45, 0]
pt2 = [0.8, 0.35, 0]

state0 = mars.reduce_from_scratch(cplx, pt1)
(state1, swaps) = mars.vineyards_123(cplx, state0, pt2)
assert len(swaps) == 0

##############################################################
import sys

sys.stdout.flush()
print()
print('Reduce from scratch', pt0)
state0 = mars.reduce_from_scratch(cplx, pt0)
sys.stdout.flush()
print()

print('vineyards to ', pt1)
(state1, swaps1) = mars.vineyards_123(cplx, state0, pt1)
sys.stdout.flush()
assert len(swaps1) > 0, 'should have swaps'
print()

print('vineyards to ', pt2)
(state2, swaps2) = mars.vineyards_123(cplx, state1, pt2)
sys.stdout.flush()
assert len(swaps2) == 0, 'should not have swaps'
print()

reduce_from_scratch: stack0.ordering: [0, 1, 2]
vineyards: stack0.ordering: [0, 1, 2]
vineyards: v_perm: [0, 1, 2]
vineyards: vine_ordering0: [0, 1, 2]

Reduce from scratch [0.8, 0.55, 0]
reduce_from_scratch: stack0.ordering: [1, 0, 2]

vineyards to  [0.8, 0.45, 0]
vineyards: stack0.ordering: [1, 0, 2]
vineyards: v_perm: [0, 1, 2]
vineyards: vine_ordering0: [1, 0, 2]

vineyards to  [0.8, 0.35, 0]
vineyards: stack0.ordering: [0, 1, 2]
vineyards: v_perm: [0, 1, 2]
vineyards: vine_ordering0: [0, 1, 2]



In [10]:
print(str(state1.stacks[0].U_t))

s1 = mars.reduce_from_scratch(cplx, pt1)
print(str(s1.stacks[0].U_t))

|×  |
|×× |
|×××|

|×  |
|×× |
|× ×|



['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']